In [61]:
from os.path import abspath
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
import os
import pandas as pd
import warnings
import json, pprint, requests, textwrap
import findspark

import json, pprint, requests, textwrap
from requests.auth import HTTPBasicAuth


findspark.init()
warnings.filterwarnings('ignore')

warehouse_location = abspath('spark-warehouse')
os.environ["JAVA_HOME"] = r'/lib/jvm/java-11-openjdk-amd64'
os.environ["SPARK_HOME"] = r"/home/guilherme/docker-livy/code/apps/spark"

os.environ["HADOOP_OPTS"]="$HADOOP_OPTS -Djava.library.path=$HADOOP_HOME/lib/native"
os.environ["LIVY_CONF_DIR"]=r"/opt/livy"




In [62]:
user = os.getenv('user')
password = os.getenv('password')
tabela = os.getenv('table')
url = os.getenv('URI_BI')


In [64]:
def executa_consulta_tabela_sql(spark: SparkSession) -> None:
    """Faz a consulta da tabela e salva em um Parquet na pasta spark-wareHouse"""

    jdbcDF = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver')\
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("dbtable", tabela) \
        .option("user", user) \
        .option("password", password)\
        .load()
    

    jdbcDF.write.mode('overwrite') \
         .saveAsTable(tabela)

In [65]:
if __name__=='__main__':
    conf = SparkConf()
    conf.set("spark.master","local[*]")
    conf.set("spark.executor.memory", "4g")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.sql.adaptive.enabled","true")
    conf.set("spark.sql.adaptive.localShuffleReader.enabled","true")
    conf.set("spark.dynamicAllocation.enabled", "false")
    conf.set("spark.sql.adaptive.optimizeSkewsInRebalancePartitions.enabled","true")
    conf.set("spark.sql.adaptive.skewJoin.enabled","true")
    conf.set("spark.sql.statistics.size.autoUpdate.enabled","true")
    conf.set("spark.sql.inMemoryColumnarStorage.compressed","true")
    conf.set("hive.exec.dynamic.partition", "true")
    conf.set("hive.exec.dynamic.partition.mode", "nonstrict")

    spark = SparkSession.builder\
            .config(conf=conf)\
            .getOrCreate()


In [66]:
df = spark.read.option("inferSchema",True) \
                 .options(header='True', inferSchema='True', delimiter=';') \
                .csv(r"/home/guilherme/docker-livy/code/apps/SEG_TIM_TREINAR_MODELO_V2_ABR_31-05_1_1.CSV")

df.createOrReplaceTempView("ProdutosView")

In [67]:
print(df.show(4))

+-----------------+-----------+------------+-------+------+--------------+-------------------+-----+-------+----+----+----------------+-------+--------------------+
|DT_REFERENCIA_TIM|       FONE|QTD_DISCADOS|QTD_ALO|VENDAS|CAMPANHA_CLARO|DT_REFERENCIA_CLARO|PLANO|SEGREAL|   X|   Y|RENDA_PER_CAPITA|DDFAIXA|NM_SUBCLUSTER_DEPARA|
+-----------------+-----------+------------+-------+------+--------------+-------------------+-----+-------+----+----+----------------+-------+--------------------+
|       2023-04-01|11910086543|           1|      1|     0|          null|               null| null|   null|null|null|            null|      B|                null|
|       2023-04-01|11910221828|           7|      4|     0|     MADURAS_A|         2023-04-01|  P�S|      2|   F|   B|             555|      B|                 A42|
|       2023-04-01|11910205181|           3|      1|     0|          null|               null| null|   null|null|null|            null|      B|                null|
|       20

In [68]:

"""Faz a Leitura do arquivo CSV"""
def reader_csv(spark: SparkSession) -> None:
    df = spark.read.option("inferSchema",True) \
                 .options(header='True', inferSchema='True', delimiter=';') \
                .csv(r"/home/guilherme/docker-livy/code/apps/SEG_TIM_TREINAR_MODELO_V2_ABR_31-05_1_1.CSV")
    

    print(df.show(10))

    df.createOrReplaceTempView("ProdutosView")
    

+-----------------+-----------+------------+-------+------+--------------+-------------------+--------+-------+----+----+----------------+-------+--------------------+
|DT_REFERENCIA_TIM|       FONE|QTD_DISCADOS|QTD_ALO|VENDAS|CAMPANHA_CLARO|DT_REFERENCIA_CLARO|   PLANO|SEGREAL|   X|   Y|RENDA_PER_CAPITA|DDFAIXA|NM_SUBCLUSTER_DEPARA|
+-----------------+-----------+------------+-------+------+--------------+-------------------+--------+-------+----+----+----------------+-------+--------------------+
|       2023-04-01|11910086543|           1|      1|     0|          null|               null|    null|   null|null|null|            null|      B|                null|
|       2023-04-01|11910221828|           7|      4|     0|     MADURAS_A|         2023-04-01|     P�S|      2|   F|   B|             555|      B|                 A42|
|       2023-04-01|11910205181|           3|      1|     0|          null|               null|    null|   null|null|null|            null|      B|              

In [ ]:
executa_consulta_tabela1(spark)

In [ ]:

host = 'http://localhost:8998'
data = {'kind': 'spark'}
headers = {'Content-Type': 'application/json'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{u'state': u'starting', u'id': 0, u'kind': u'spark'}

{'state': 'starting', 'id': 0, 'kind': 'spark'}

In [ ]:
session_url = host + r.headers['location']
r = requests.get(session_url, headers=headers)
r.json()

{u'state': u'idle', u'id': 0, u'kind': u'spark'}

{'state': 'idle', 'id': 0, 'kind': 'spark'}

In [ ]:
statements_url = session_url + '/statements'
data = {'code': '1 + 1'}
r = requests.post(statements_url, data=json.dumps(data), headers=headers)
r.json()

{u'output': None, u'state': u'running', u'id': 0}

{'output': None, 'state': 'running', 'id': 0}

In [ ]:
                            
host = "http://localhost:8998"

#Documentação https://livy.incubator.apache.org/docs/latest/rest-api.html#session


sessions_url = host + "/sessions"


auth = HTTPBasicAuth("***", "***")

headers = {'Content-Type': 'application/json'}

data = {
        'from': 0, 
        'size': 10
       }
r = requests.get(sessions_url, data=json.dumps(data), headers=headers, auth=auth, verify=False)

response_body = r.json()

print("Sessions fetched starting ", response_body['from'])
print("Number of session fetched", response_body['total'])
session_list = response_body['sessions']

for session in session_list:
    print("The session {0} has a state {1} ".format(session['id'],session['state']))

Sessions fetched starting  0
Number of session fetched 2
The session 0 has a state idle 
The session 1 has a state idle 


In [ ]:
#Create a new Spark session
data = {
    'kind':'pyspark'
}

r = requests.post(sessions_url, data=json.dumps(data), headers=headers, auth=auth, verify=False)
response_body = r.json()

session_id = response_body['id']
session_state = response_body['state']
print("Spark session {0} created. Current state is {1}".format(session_id,session_state))
created_session_url = r.headers['location']

Spark session 2 created. Current state is starting


In [ ]:
#Query information about the session we just created
this_session_url = host + created_session_url
print("this_session_url", this_session_url)

r = requests.get(this_session_url, headers=headers,auth=auth, verify=False)
pprint.pprint(r.json())

this_session_url http://localhost:8998/sessions/2
{'appId': None,
 'appInfo': {'driverLogUrl': None, 'sparkUiUrl': None},
 'id': 2,
 'kind': 'pyspark',
 'log': ['23/06/21 04:58:30 INFO Utils: Successfully started service '
         "'org.apache.spark.network.netty.NettyBlockTransferService' on port "
         '40609.',
         '23/06/21 04:58:30 INFO NettyBlockTransferService: Server created on '
         '45dab089587a:40609',
         '23/06/21 04:58:30 INFO BlockManager: Using '
         'org.apache.spark.storage.RandomBlockReplicationPolicy for block '
         'replication policy',
         '23/06/21 04:58:30 INFO BlockManagerMaster: Registering BlockManager '
         'BlockManagerId(driver, 45dab089587a, 40609, None)',
         '23/06/21 04:58:30 INFO BlockManagerMasterEndpoint: Registering block '
         'manager 45dab089587a:40609 with 366.3 MB RAM, BlockManagerId(driver, '
         '45dab089587a, 40609, None)',
         '23/06/21 04:58:30 INFO BlockManagerMaster: Registered

In [72]:
#Execute code interactively in this session using session/<ID>/statements interface

statements_url = this_session_url + "/statements"
pprint.pprint(statements_url)

data = {
  'code': "11+11"
}

r = requests.post(statements_url, data=json.dumps(data), headers=headers, auth=auth, verify=False)
print("Respone status code" ,r.status_code )
print("Response received is ",r.json())
print("Poll URI is ",r.headers['location'])
#response_body = r.json()

'http://localhost:8998/sessions/2/statements'
Respone status code 201
Response received is  {'id': 2, 'code': '11+11', 'state': 'waiting', 'output': None, 'progress': 0.0, 'started': 0, 'completed': 0}
Poll URI is  /sessions/2/statements/2


In [73]:
# Wait a while before executing this cell. Spark session start up takes time to run your code.
specific_statement = host + r.headers['location']
print("monitoring url is ", specific_statement)

r = requests.get(specific_statement, headers=headers, auth=auth, verify=False)
print("Response status is ",r.status_code)
print("Response received is ", pprint.pprint(r.json()))

monitoring url is  http://localhost:8998/sessions/2/statements/2
Response status is  200
{'code': '11+11',
 'completed': 1687323629800,
 'id': 2,
 'output': {'data': {'text/plain': '22'}, 'execution_count': 2, 'status': 'ok'},
 'progress': 1.0,
 'started': 1687323629799,
 'state': 'available'}
Response received is  None


In [ ]:
#Envia arquivos Python para executar no Cluster do Spark



batch_url = host + "/batches"
print("batch_url", batch_url)

data = {
    'file' : '/home/guilherme/docker-livy/submit_teste_01.py'    
}

r = requests.post(batch_url, data=json.dumps(data), headers=headers, auth=auth, verify=False)
returned_batch_url = r.headers
print("Respone status code" , r.status_code)
print("Poll URI is ",returned_batch_url )
print("Response is ", pprint.pprint(r.json()))

In [ ]:
#Check results of executed code
specific_batch = host + returned_batch_url
print("specific batch request ",specific_batch)

r = requests.get(specific_batch,headers=headers, auth=auth, verify = False)
print("Response status is ",r.status_code)
print("Response received is ", pprint.pprint(r.json()))

In [ ]:
#Envia Codigo Para Cluster
data = {
  'code': textwrap.dedent("""
    import random
    NUM_SAMPLES = 100000
    def sample(p):
      x, y = random.random(), random.random()
      return 1 if x*x + y*y < 1 else 0

    count = sc.parallelize(xrange(0, NUM_SAMPLES)).map(sample).reduce(lambda a, b: a + b)
    print "Pi is roughly %f" % (4.0 * count / NUM_SAMPLES)
    """)
}

r = requests.post(statements_url, data=json.dumps(data), headers=headers)
pprint.pprint(r.json())

{u'id': 12,
u'output': {u'data': {u'text/plain': u'Pi is roughly 3.136000'},
            u'execution_count': 12,
            u'status': u'ok'},
u'state': u'running'}

In [ ]:
data = {'kind': 'sparkr'}
r = requests.post(host + '/sessions', data=json.dumps(data), headers=headers)
r.json()

{u'id': 1, u'state': u'idle'}

In [ ]:
data = {
  'code': textwrap.dedent("""
    n <- 100000
    piFunc <- function(elem) {
      rands <- runif(n = 2, min = -1, max = 1)
      val <- ifelse((rands[1]^2 + rands[2]^2) < 1, 1.0, 0.0)
      val
    }
    piFuncVec <- function(elems) {
      message(length(elems))
      rands1 <- runif(n = length(elems), min = -1, max = 1)
      rands2 <- runif(n = length(elems), min = -1, max = 1)
      val <- ifelse((rands1^2 + rands2^2) < 1, 1.0, 0.0)
      sum(val)
    }
    rdd <- parallelize(sc, 1:n, slices)
    count <- reduce(lapplyPartition(rdd, piFuncVec), sum)
    cat("Pi is roughly", 4.0 * count / n, "\n")
    """)
}

r = requests.post(statements_url, data=json.dumps(data), headers=headers)
pprint.pprint(r.json())

{u'id': 12,
 u'output': {u'data': {u'text/plain': u'Pi is roughly 3.136000'},
             u'execution_count': 12,
             u'status': u'ok'},
 u'state': u'running'}

In [ ]:
"""Submit"""
'''
./bin/spark-submit \
  --master spark://207.184.161.138:7077 \
  /home/guilherme/docker-livy/submit_teste_01.py \
  1000
'''

In [ ]:
"""Submit"""
'''
./bin/spark-submit \
  --master spark://207.184.161.138:7077 \
  /home/guilherme/docker-livy/submit_teste_02.py \
  1000
'''